<h1>Buscando dados da internet</h1>

<h3>Laboratório de Coletas, Preparação e Análise de Dados.</h3>
Desenvolvendo um crawler básico.

Prof. Luan Garcia

Baseado no material de Richard Mitchel
https://github.com/REMitchell/python-scraping

Com a biblioteca Beautiful Soup vimos como é possível tratar documentos HTML como objetos no modelo DOM, buscando informação pelas *tags* e seus atributos. Porém, para chegarmos ao ponto de processar um documento HTML precisamos realizar o download do documento a partir do seu servidor web e como descobrir novos documentos a partir de um link de origem.

Neste notebook, veremos como é possível fazer requisições HTTP utilizando a biblioteca urllib e como lidar com alguns problemas básicos que podem acontecer quando estamos trabalhando com um crawler.


Primeiro passo é importar o módulo request da biblioteca urllib. Não é necessário instalar, pois é uma biblioteca nativa do Python.

Para fazer o download do documento web, basta sabermos o endereço do documento e chamar o método read() do **urlopen**.

Para podermos tratar o documento utilizando a biblioteca BeautifulSoup podemos passar diretamente o documento carregado através da requisição HTTP e depois manipular conforme o necessário.

Um problema bastante comum é o site que estamos tentando acessar não estar disponível, seja porque erramos o endereço, seja porque o servidor está fora do ar.

Um solução simples é garantir que tenhamos um documento para tratar utilizando exceções.

O objetivo de um crawler é descobrir, a partir de um link raiz, outros documentos que estejam linkados de alguma forma.

Uma maneira simples de realizar isto é filtrando pelas tags de links de um documento HTML. Com isto, encontramos todos os links dentro de um documento.

Abaixo, um exemplo utilizando como raiz a página do ator Kevin Bacon na Wikipedia.

Evidentemente, nem todos links serão de nosso interesse. Podemos filtrar apenas os links que nos interessam procurando por algum padrão no endereço e utilizar uma expressão regular para realizar o filtro.

Abaixo, vamos filtrar apenas links para outros artigos da wiki, ignorando âncoras, links para arquivos, etc. Faremos isso nos aproveitando de conhecimento de como um verbete na wiki é organizado. Todos links de artigos estarão sempre dentro da tag **div** que contém um atributo de **id** com valor **'bodyContent'**. Além disso, todo link de verbete necessariamente começa com o endereço "/wiki/" e não possui ":" no endereço.

Podemos generalizar este código em forma de uma função **getLinks()**. Isto possibilitará que busquemos os links de qualquer verbete da wiki.

A função **getLinks()** anterior funciona se quisermos encontrar todos os links de uma única página, porém, se quisermos fazer um crawler efetivo, precisamos procurar por páginas linkadas dentro de outras páginas de forma recursiva. Podemos fazer isso chamando a nosa própria função de procurar link de forma recursiva.

Experimente executar o comando a seguir e veja o que acontece. 

O que aconteceu? Será que isto é um problema? Se for, como podemos solucioná-lo?

In [8]:
from urllib.request import urlopen

In [24]:
import requests
from bs4 import BeautifulSoup

def pega_autores(link):
    url = "https://www.imdb.com"+link
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }


    try:
        response_serie = requests.get(url, headers=headers)  # Corrigindo a passagem da URL
        response_serie.raise_for_status()
    except requests.HTTPError as e:
        print("The server returned an HTTP error:", e)
        return None
    except requests.RequestException as e:
        print("Request error:", e)
        return None
    else:
        html_serie = response_serie.text
        soup = BeautifulSoup(html_serie, 'html.parser')
        lista_div = soup.find('div', class_="ipc-chip-list__scroller")
        lista_span = lista_div.find_all('span', class_="ipc-chip__text")
        print("Generos: ")
        for span in lista_span:
            print(span.text)
        list_atores = soup.find_all('a', class_="sc-bfec09a1-1 gCQkeh")
        print("Atores: ")
        for ator in list_atores:
           print(ator.text)
        popularidade = soup.find('div', class_='sc-5f7fb5b4-1 fTREEx')
        print("Popularidade:", popularidade.text)
    



            

In [25]:
import requests
from bs4 import BeautifulSoup
import re   

url = "https://www.imdb.com/chart/toptv/?ref_=nv_tvv_250"

# Adicionando um cabeçalho de usuário
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
}

try:
    response = requests.get(url, headers=headers)
    response.raise_for_status()
except requests.HTTPError as e:
    print("The server returned an HTTP error:", e)
except requests.RequestException as e:
    print("Request error:", e)
else:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    lista_series = soup.find('ul', class_="ipc-metadata-list ipc-metadata-list--dividers-between sc-a1e81754-0 eBRbsI compact-list-view ipc-metadata-list--base")
    if lista_series:
        series = lista_series.find_all('li')
        
    for serie in series:
        titulo_elemento = serie.find(class_= "ipc-title__text").text
        print(titulo_elemento)
        ano_estreia_elemento = serie.find(class_="sc-b0691f29-8 ilsLEX cli-title-metadata-item").text
        div = soup.find('div', class_="sc-b0691f29-7 hrgukm cli-title-metadata")
        spans = div.find_all('span', class_="sc-b0691f29-8 ilsLEX cli-title-metadata-item")
        
        if len(spans)>= 2:
            numero_episodios = spans[1].text
            print(numero_episodios)
            
        rating = serie.find(class_= "ipc-rating-star").text
        link_pagina_filme = serie.find("a", class_= "ipc-title-link-wrapper")
        re2_result = re.search(r'\d{4}',ano_estreia_elemento)
        
        if re2_result:
            ano_estreia_elemento = re.search(r'\d{4}',ano_estreia_elemento).group(0)
            print(ano_estreia_elemento)
            
        re_result = re.search(r'^\d\.\d',rating) #aplicando a regex para extrair apenas o padrão 'digito ponto digito'
        
        if re_result:
            rating = re.search(r'^\d\.\d',rating).group(0) #string resultante da regex
            print(rating)
            
        link = link_pagina_filme["href"]
        print(link)
        pega_autores(link)






1. Breaking Bad
62 eps
2008
9.5
/title/tt0903747/?ref_=chttvtp_t_1
Generos: 
Crime
Drama
Thriller
Atores: 
Bryan Cranston
Aaron Paul
Anna Gunn
Betsy Brandt
RJ Mitte
Dean Norris
Bob Odenkirk
Steven Michael Quezada
Jonathan Banks
Giancarlo Esposito
Charles Baker
Jesse Plemons
Christopher Cousins
Laura Fraser
Matt Jones
Michael Shamus Wiles
Lavell Crawford
Ray Campbell
Popularidade: 28
2. Planeta Terra II
62 eps
2016
9.5
/title/tt5491994/?ref_=chttvtp_t_2
Generos: 
Documentary
Atores: 
David Attenborough
Chadden Hunter
Michael J. Sanderson
Ed Charles
Sandesh Kadur
Max Hug Williams
Fredi Devas
Elizabeth White
Toby Nowlan
Barrie Britton
Gordon Buchanan
Mark MacEwen
Thomas Crowley
Emma Brennand
Pete McCowen
Rob Drewett
Jerome Poncet
Emma Napper
Popularidade: 1,399
3. Planeta Terra
62 eps
2006
9.4
/title/tt0795176/?ref_=chttvtp_t_3
Generos: 
Documentary
Family
Atores: 
Sigourney Weaver
David Attenborough
Nikolay Drozdov
Thomas Anguti Johnston
Sanae Ueda
Huw Cordey
Mark Linfield
Doug Allan
Cha

KeyboardInterrupt: 